# Retail Demo Store Experimentation Workshop - Amazon CloudWatch Evidently

In this workshop we will create an A/B experiment using [Amazon CloudWatch Evidently](https://aws.amazon.com/cloudwatch/features/). Evidently lets application developers conduct experiments and identify unintended consequences of new features before rolling them out for general use, thereby reducing risk related to new feature roll-out. Evidently allows you to validate new features across the full application stack before release, which makes for a safer release. When launching new features, you can expose them to a small user base, monitor key metrics such as page load times or conversions, and then dial up traffic. Evidently also allows you to try different designs, collect user data, and release the most effective design in production. 

Recommended Time: 30 minutes

## CloudWatch Evidently concepts

To get started with CloudWatch Evidently, for either a feature launch or an A/B experiment, you first create a project. A project is a logical grouping of resources. Within the project, you create features that have variations that you want to test or launch.

When the Retail Demo Store project was deployed in your account, an Evidently project and several Evidently features were automatically created. These resources are defined in the [evidently.yaml](https://github.com/aws-samples/retail-demo-store/blob/master/aws/cloudformation-templates/base/evidently.yaml) CloudFormation stack. Open a new browser window/tab and browse to [CloudWatch](https://console.aws.amazon.com/cloudwatch/home); under "Application monitoring" in the left navigation, you will find Evidently.

![Evidently Project page](./images/evidently/evidently_project.png)

### Retail Demo Store features

When a user is interacting with web application views like the home page, product detail page, the search auto-complete drop-down, and the shop livestreams page, the web application will make requests to the Recommendations microservice to retrieve or order products for the given user experience. One of the parameters passed with each of these requests is the feature name. The feature name is mapped to Evidently features. The following application features are already instrumented in the Retail Demo Store web application as well as mapped to Evidently features in the Evidently project.

- **Home page (top)**: the top of the home page view displays recommended products in two states: when the current visitor/user is new/cold and when the current visitor is warm. The new/cold visitor/user is automatically transitioned to the warm experience after 2 product view events have been sent to the Personalize event tracker. Selecting an existing shopper also puts the user into the warm user experience.
    - "**Popular products**": for new/cold users, displays popular products in a grid widget. The feature name is `home_product_recs_cold`.
    - "**Inspired by your shopping trends**": for existing/warm users, displays products from one of the supported product recommenders configured in the Recommendations microservice. The feature name is `home_product_recs`.
- **Home page (bottom)**: the bottom of the home page view displays featured products in a carousel widget.
    - "**Featured products**": this is the carousel widget at the bottom of the home page view where featured products are displayed. The feature name is `home_featured_rerank`.
- **Product detail page**: when you click on a product, you are taken to the product detail view.
    - "**Compare similar items**": this the carousel widget on the product detail view that displays products similar to the product being displayed. The feature name is `product_detail_related`.
- **Navigation**: the header navigation.
    - "**Search results**": this is the search drop-down in the web application's navigational header. For this feature, we can test personalized ranking of search results against search results that are ordered by Amazon OpenSearch. The feature name is `search_results`.
- **Shop live streams**: on the "Shop" drop-down is a "Shop Livestreams" option. This page provides a live streaming interface for demonstrating products and making recommendations.
    - "**Shop livestreams - discounted products**": this is the sidebar vertical widget on the Shop Livestreams page. It displays discounted products highighted in the live stream.  The feature name is `live_stream_prod_recommendation`.
    - "**Shop livestreams - Compare similar items**": this is the carousel widget at the bottom of the Shop Livestreams page. It displays products similar to the product currently being featured in the live stream. The feature name is `live_stream_prod_discounts`.

To examine the Evidently features, click on the Retail Demo Store's project in Evidently.

![Evidently Features page](./images/evidently/evidently_features.png)

## Retail Demo Store / CloudWatch Evidently integration

Let's examine the Evidently integration in more detail. 

- You start by creating features in Evidently mapped to user experiences in your application that you want to control with launch/feature flags or experiments. As mentioned above, this was already done for you in the Retail Demo Store during deployment. User experiences can be as simple as a view title or button color or as complex as a panel or widget.
- Next you instrument the user interface logic in your application for each feature to call Evidently to determine the variation to use. Each feature can have one or more variations and each variation has a value that can be a boolean/flag, number, or string. In our case, we're storing small JSON snippets as string variation values. The JSON contains the details needed to map a variation value to a recommender implementation.
- Experiments can be created for features by specifying the experiment duration, how to split traffic across variations, metrics that will be collected to measure the outcome of the experiment, and more.
- Your application calls the Evidently [EvaluateFeature](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_EvaluateFeature.html) or [BatchEvaluateFeature](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_BatchEvaluateFeature.html) API to retrieve the variation details for given feature(s) for the current user. This can be done in the client using the AWS SDK for JavaScript or server-side using any of the supported language SDKs. For the Retail Demo Store, these calls are made in the Recommendations microservice. 
   - If an experiment is active for a feature, the evaluate feature response will include a `reason` of `EXPERIMENT_RULE_MATCH`. In this case, you also need to send an experiment exposure event to Evidently to indicate that the current user is receiving the assigned variation.
- When/if the user "converts" against a variation that is part of an experiment, a "conversion" event needs to be sent to Evidently. In our case, a conversion is when the user clicks on a product included in a variation's recommended products. The Retail Demo Store web application calls the `/experiment/outcome` endpoint on the Recommendations microservice which then calls the Evidently [PutProjectEvents](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_PutProjectEvents.html) API.

The following architecture diagram summarizes the integration.

![Retail Demo Store Evidently architecture](./images/evidently/rds_evidently_architecture.png)


## Running an Evidently experiment

Let's walk through the process of creating an experiment in Evidently, examine how the Retail Demo Store web application displays variations, and then simulate an experiment so we can inspect the results. We're going to illustrate how to run an experiment in code using the Python programming language but you could also use the Evidently console.

### Our experiment hypothesis

**Sample scenario:**

Website analytics have shown that user sessions frequently end on the home page for our e-commerce site, the Retail Demo Store. Furthermore, when users do make a purchase, most purchases are for a single product. Currently on our home page we are using a basic approach of recommending featured products (control experience). We hypothesize that replacing the current featured products approach on the homepage with personalized recommendations from Amazon Personalize (variation) will result in increasing the CTR (click-through rate) of products by 25%. The current click-through rate is 15%.

### Setup - import dependencies

First, let's import the dependencies needed to interact with the Evidently API via Python.

In [ ]:
import boto3
import json
from datetime import datetime, timedelta

Next let's create the clients we'll need to make API calls. We'll also declare the feature name that we'll be using for this experiment (`home_product_recs`).

In [ ]:
# Evidently client
evidently = boto3.client('evidently')
# Service discovery will allow us to dynamically discover Retail Demo Store microservices
servicediscovery = boto3.client('servicediscovery')

feature = 'home_product_recs'

# The Uid is a unique ID and we need it to find the role made by CloudFormation
with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        project_name = tag['Value']
        break

print('project_name:', project_name)

### Inspect feature

Before creating and starting our experiment, let's take a look at the feature. As a reminder, this feature was setup in Evidently during the deployment of the Retail Demo Store.

In [ ]:
response = evidently.get_feature(project = project_name, feature = feature)
print(json.dumps(response['feature'], indent = 2, default = str))

In the response above, take special note of the `variations` array and the `value.stringValue` value for each variation. As mentioned above, we're actually storing a short JSON document as the `stringValue`. The Recommendations microservice will parse this string as JSON and extract the information it needs to "wire up" each variation to a product recommendation implementation. In this case we have two variations for this feature:

- Variation #1: **Featured Products** - displays featured products from the product catalog. For this variation, we just need the `type` of `product` to tell the Recommendations service how to resolve featured products from the Products microservice.
- Variation #2: **Personalize-UserPersonalization** - displays personalized product recommendations from an Amazon Personalize campaign or recommender. To resolve this variation we need a `type` of `personalize-recommendations` and the Personalize campaign or recommender ARN to call to get recommendations for each user. The `arn` field in the JSON snippet provides the ARN needed by the Recommendations service.

### Evaluate feature before experiment

Before creating an experiment, let's take a look at what the [EvaluateFeature](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_EvaluateFeature.html) API returns for the `home_product_recs` feature.

In [ ]:
user_id = '123'

response = evidently.evaluate_feature(
    entityId = user_id,
    feature = feature,
    project = project_name
)

print(json.dumps(response, indent = 2, default = str))

Take note that the `reason` is `DEFAULT` and the only variation and value is the default variation from the `GetFeature` call above. This means that there is currently not an active experiment or launch for this feature.

### Create experiment

Now we will create an experiment for the `home_product_recs` feature by calling the [CreateExperiment](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_CreateExperiment.html) API. You could also create the experiment in the Evidently console.

First we will define the metric event pattern. This pattern is used match events for exposures and outcome/conversion events across all features back to this particular experiment. This is very similar to Amazon EventBridge [patterns](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-event-patterns.html). The logic in the Recommendations service is to use `userDetails.userId` to identify the user and to build the `valueKey` by taking the feature name, convert it from snake case to camel case, and append `Clicked` to the result. So `home_product_recs` becomes `homeProductRecsClicked`. This is put within a `details` dictionary to arrive at a final value key of `details.homeProductRecsClicked`. Experiments for every feature in the Retail Demo Store is handled the same way.

To hopefully pull this together, here is an example of a conversion event that the Recommendations service will send to Evidently for user `abc123`. Since we're tracking click-through-rate, the value is `1.0` to indicate a conversion/click for this user.

```javascript
{
    'details': {
        'homeProductRecsClicked': 1.0
    },
    'userDetails': {
        'userId': 'abc123'
    }
}
```

Now the pattern that Evidently needs to match events formatted like this is defined in the following cell.

In [ ]:
metric_event_pattern = {
    "userDetails.userId": [
        {
            "exists": True
        }
    ],
    "details.homeProductRecsClicked": [
        {
            "exists": True
        }
    ]
}

Now we can create the experiment, passing the metric event pattern in the `metricGoals`.

In [ ]:
experiment_name = 'home_product_recs_personalization'

response = evidently.create_experiment(
    project = project_name,
    name = experiment_name,
    description = 'Testing default product recommendations of featured products against Amazon Personalize generated recommendations',
    metricGoals = [
        {
            'desiredChange': 'INCREASE',
            'metricDefinition': {
                'entityIdKey': 'userDetails.userId',
                'valueKey': 'details.homeProductRecsClicked',
                'eventPattern': json.dumps(metric_event_pattern),
                'name': 'homeProductRecsClicked'
            }
        },
    ],
    onlineAbConfig={
        'controlTreatmentName': 'FeaturedProducts',
        'treatmentWeights': {
            'FeaturedProducts': 50000,
            'Personalize-UserPersonalization': 50000
        }
    },
    samplingRate = 100000,
    treatments=[
        {
            'feature': feature,
            'variation': 'FeaturedProducts',
            'name': 'FeaturedProducts'
        },
        {
            'feature': feature,
            'variation': 'Personalize-UserPersonalization',
            'name': 'Personalize-UserPersonalization'
        }
    ]
)

print(json.dumps(response['experiment'], indent = 2, default = str))

### Start experiment

With the experiment created, now it's time to start the experiment to make it active. We'll set the experiment end date to be one week from now.

In [ ]:
experiment_end_date = datetime.now() + timedelta(days = 7)

response = evidently.start_experiment(
    project = project_name,
    experiment = experiment_name,
    analysisCompleteTime = experiment_end_date
)

print(json.dumps(response, indent = 2, default = str))

### Evaluate feature after experiment has started

Now let's take a look at the EvaluateFeature API response now that the experiment has been created and started.

In [ ]:
response = evidently.evaluate_feature(
    entityId = user_id,
    feature = feature,
    project = project_name
)

print(json.dumps(response, indent = 2, default = str))

Notice this time that the `reason` is now `EXPERIMENT_RULE_MATCH`, we have details on the active experiment in the `details` dictionary, and we have details on the assigned variation in the `value` dictionary. The Recommendations service uses this response to know which resolver to use to provide the response.

### Inspect storefront

To see the experiment in action on the Retail Demo Store storefront, browse to the storefront deployed in your account, and then either sign in or create an account. On the home view you should see an indicator that an experiment is active and the variation assigned to the currently selected shopper is indicated for each product.

![Retail Demo Store active Evidently experiment](./images/evidently/rds_active_evidently_experiment.png)

To see how the assigned variation changes depending on the current shopper, try switching shoppers from the "Shopper" dropdown in the top naviation.

![Retail Demo Store switch shopper](./images/evidently/rds_switch_shoppers.png)

You may have to cycle through a few shoppers to land on one that hashes to a different variation.

### Create conversion/click events

To trigger a conversion event to be sent to Evidently's [PutProjectEvents](https://docs.aws.amazon.com/cloudwatchevidently/latest/APIReference/API_PutProjectEvents.html) API, click on a product displayed by the variation. As you switch shoppers, trigger conversion events for some and not others to see how they're tracked in the Evidently results dashboard for the experiment.

## Next Steps

You have completed the exercise for implementing an A/B test using the Amazon CloudWatch Evidently. From here you can try one of the other experimentation workshops.

Since Evidently features are evaluated before the built-in experimentation types, **be sure to cancel your Evidently experiment**. This can be done in the CloudWatch > Evidently console. 